# Домашнє завдання: Інтерактивні візуалізації з Plotly

## Опис завдання
У цьому домашньому завданні ви будете створювати інтерактивні візуалізації з допомогою бібліотеки Plotly. Ви дізнаєтесь різницю між Plotly Express (швидкі графіки) та Graph Objects (повний контроль), та створите інтерактивний дашборд.

**Опис колонок:**
- `datetime` - дата та час
- `season` - сезон (1=весна, 2=літо, 3=осінь, 4=зима)
- `holiday` - чи є день святковим (0=ні, 1=так)
- `workingday` - чи є день робочим (0=ні, 1=так)
- `weather` - погодні умови (1=ясно, 2=туман, 3=легкий дощ, 4=сильний дощ)
- `temp` - температура в градусах Цельсія
- `atemp` - відчувається як температура
- `humidity` - вологість (%)
- `windspeed` - швидкість вітру
- `casual` - кількість випадкових користувачів
- `registered` - кількість зареєстрованих користувачів
- `count` - загальна кількість орендованих велосипедів

## Підготовка даних


In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Завантаження даних
df = pd.read_csv('../data/yulu_rental.csv')
df['datetime'] = pd.to_datetime(df['datetime'])

# Для plotly краще не встановлювати datetime як індекс
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['hour'] = df['datetime'].dt.hour
df['weekday'] = df['datetime'].dt.day_name()

# Додаємо назви сезонів
season_map = {1: 'Весна', 2: 'Літо', 3: 'Осінь', 4: 'Зима'}
df['season_name'] = df['season'].map(season_map)

## Завдання 1: Базовий інтерактивний лінійний графік (Plotly Express)

**Завдання:**
Створіть інтерактивний лінійний графік динаміки оренди за часом (рівень деталізації - як в даних) з можливістю zoom та hover.

Дайте відповіді на питання.
**Питання для інтерпретації:**
1. Яка перевага інтерактивного графіка над статичним?
2. Чому на графіку є "пробіли" - ділянки, де одна пряма лінія зʼєднує два "суцільних" блоки з даними? Як би ви це могли дослідити на статичному графіку?


In [2]:
fig = px.line(df, x='datetime', y='count',
              title='Інтерактивний графік динаміки оренди велосипедів')
fig.update_traces(line_width=2)
fig.update_layout(
    hovermode='x unified',
    template='plotly_white'
)

1) З інтерактивним графіком можна взаємодіяти, наближувати окремі ділянки для деталізації, виводити додаткову інформацію в hover
2) На графіку є пробіли, тому що у вхідних даних присутні дані лише за перші 19 днів кожного місяця. На статичному графіку, побудованому в Pandas, дані за ці дні приймали значення 0

## Завдання 2: Scatter plot з додатковими даними (Plotly Express)

**Завдання:**
Створіть scatter plot кількості орендованих велосипедів випадковими користувачами vs кількості орендованих велосипедів зареєстрованими користувачами. Розмір точок встановіть за сумарною кількістю велосипедів, які були взяті в оренду, а колір - за сезоном. В hover_data - додайте деталі, які допоможуть вам в подальшому аналізі.

Дослідіть графік. Зверніть увагу, що ви можете вмикати і вимикати окремі сезони, якщо будете клікати на колір сезону в легенді графіку.

**Дайте відповідь на питання.**
- Як ви проінтерпретуєте роздвоєність цього графіку (дві явні лінії)? Що це означає?
- Які висновки для компанії, які дає велосипеди в оренду, ви можете зробити з цього графіку? 3 основних висновки.

In [3]:
fig = px.scatter(df,
                 x='casual',
                 y='registered',
                 size='count',
                 color='season_name',
                 hover_data=['datetime', 'season_name', 'count'],
                 title='<b>Аналіз кількості орендованих велосипедів зареєстрованими і випадковими користувачами<b>',
                 labels={'season_name': 'Сезон',
                         'casual': 'Випадкові користувачі',
                         'registered': 'Зареєстровані користувачі',
                         'datetime': 'Дата та час'}
                )
fig.update_layout(
    height=500,
    template='plotly_white',
    title_x=0.5
    )
fig.show()

1. Роздвоєність графіку може бути пояснена поведінкою користувачів в робочі і вихідні дні. В робочі дні кількість оренд вища і більша частина користувачів це зареєстровані користувачі. Якщо подивитись додаткову інформацію по точкам, то в більшості це або 8ма ранку, або 17-18та вечора, тобто час коли люди їдуть на роботу, та з роботи. А у вихідні дні кількість зареєстрованих користувачів менша, проте кількість випадкових може сягати більше 350 на годину. Це чітко видно, якщо зробити кольорове забарвлення не по сезонах, а по типу дня `workingday`.

2. Основні висновки:
- Зареєстровані користувачі в цілому більше орендують велосипеди - максимальна точка 977 на годину, в той час, як для випадкових користувачів втричі менше. Тобто слід звернути увагу на зареєстрованих користувачів і стимулювати нових користувачів до реєстрації

- Так як робочих днів в цілому більше і кількість оренд на годину в такі дні вище, то люди в основному реєструються і орендують велосипеди, як засіб доїхати на роботу. Можна збільшувати кількість велосипедів доступних до оренди в пікові години - 8 ранку та 17-18 вечора в будні, а також розміщувати їх в місцях основних потоків людей, наприклад там, де розміщуються основні підприємства. У вихідні дні раціонально розміщувати велосипеди біля основних визначних місць, пам'яток та парків.

- Так як існує тенденція до роздвоєння поведінки користувачів, то можна використовувати різні бізнес-моделі для зареєстрованих користувачів і для випадкових, наприклад робити вищу плату для випадкових користувачів у вихідні.

## Завдання 3: Порівняння Plotly Express vs Graph Objects

**Завдання:**
Створіть лінійний графік помісячної динаміки оренди велосипедів двома способами - з Plotly Express та з Graph Objects.

**Дайте відповіді на питання.**
1. Як ви розумієте основну різницю між цими двома підходами?
2. Коли краще використовувати Plotly Express?
3. Коли потрібен Graph Objects?


In [4]:
monthly_data = df.groupby(['year', 'month'])['count'].sum().reset_index()

month_names_ua = {
    1: 'Січ', 2: 'Лют', 3: 'Бер', 4: 'Кві', 5: 'Тра', 6: 'Чер',
    7: 'Лип', 8: 'Сер', 9: 'Вер', 10: 'Жов', 11: 'Лис', 12: 'Гру'
}

monthly_data['month_year'] = monthly_data.apply(
    lambda row: f"{month_names_ua[row['month']]} {row['year']}", axis=1
)

In [5]:

fig = px.line(monthly_data,
              x='month_year',
              y='count', 
              title='<b>Помісячна динаміка оренди велосипедів</b> Plotly Express',
              labels={'count': 'Кількість орендованих велосипедів',
                      'month_year': 'Місяць'}
              )

fig.update_traces(line_width=2,
                  mode='lines+markers')
fig.update_layout(title_x=0.5,
                  hovermode='x unified',
                  template='plotly_white')
fig.show()

In [6]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=monthly_data['month_year'],
        y=monthly_data['count'],
        mode='markers+lines'
    )
)

fig.update_layout(
    title='<b>Помісячна динаміка оренди велосипедів</b> Graph Objects',
    title_x=0.5,
    xaxis_title='Місяць',
    yaxis_title='Кількість орендованих велосипедів',
    hovermode='x unified',
    template='plotly_white'
)

fig.show()

1. Plotly Express швидше, простіше, добре працює з Pandas DataFrame, але Graph Objects має більше можливостей керування, але й є складнішим завдяки цьому.
PS: для цього прикладу вбудований формат hoverв go мені подобається більше, але розумію, що для більш складного графіка в go потрібно буде робити окремі налаштування.
2. Plotly Express краще використовувати, коли не потрібні нестандартні налаштування, а потрібно швидко побачити результат
3. Graph Objects краще використовувати,.. коли потрібно забезпечити гнучкість, зробити тонкі налаштування, відобразити декілька графіків на одній фігурі

## Завдання 4: Дашборд з make_subplots (Graph Objects)

**Завдання:**
Створіть дашборд з 4 різними графіками в одній фігурі:
- Bar chart - середні значення загальної кількості оренд велосипедів за сезонами
- Pie chart - відсоткове співвідношення погодних умов в даних
- Line chart - середнє значення загальної кількості оренд велосипедів за годинами протягом доби
- Scatter plot - кореляція температури vs вологість

Додайте заголовок на дашборд.

**Дайте відповідь на питання**
- На ваш погляд, яка перевага об'єднання графіків в один дашборд?

In [7]:
fig_d = make_subplots(
    rows=2, cols=2,
    specs=[
        [{"type": "xy"}, {"type": "domain"}], #domain для Pie
        [{"type": "xy"}, {"type": "xy"}]
    ],
    subplot_titles=('Середнє значення оренд за сезонами', 'Розподіл погодних умов',
                    'Середнє значення оренд за годинами', 'Температура vs Вологість')
)

# Bar chart - середні значення загальної кількості оренд велосипедів за сезонами
season_avg = df.groupby('season')['count'].mean()

fig_d.add_trace(
    go.Bar(
        x=season_avg.index,
        y=season_avg.values,
        marker_color='lightblue',
        hovertemplate='Середня кількість: %{y:.0f}<extra></extra>'
    ),
    row=1, col=1
)

# Pie chart - відсоткове співвідношення погодних умов в даних
weather_names = {1: 'Ясно', 2: 'Туман',  3: 'Легкий дощ', 4: 'Сильний дощ'}
df['weather_name'] = df['weather'].map(weather_names)
weather_counts = df['weather_name'].value_counts()

fig_d.add_trace(
    go.Pie(
        labels=weather_counts.index,
        values=weather_counts.values,
        textinfo='label+percent',
        texttemplate='%{label}<br>%{percent:.2%}',
        marker=dict(
        colors=["#EDF879", "#BD86EA", "#464CEC", "#0F0578"]
    )
    ),
    row=1, col=2
)

# Line chart - середнє значення загальної кількості оренд велосипедів за годинами протягом доби
hour_avg = df.groupby('hour')['count'].mean()

fig_d.add_trace(
    go.Scatter(
        x=hour_avg.index,
        y=hour_avg.values,
        mode='lines+markers',
        hovertemplate='Година: %{x}:00<br>Середня кількість: %{y:.0f}<extra></extra>'
    ),
    row=2, col=1
)

# Scatter plot - кореляція температури vs вологість
fig_d.add_trace(
    go.Scatter(
        x=df['temp'],
        y=df['humidity'],
        mode='markers',
        marker=dict(
            color=df['season'],
            colorscale='viridis',
            colorbar=dict(title='Сезон',
                          tickvals=[1, 2, 3, 4],
                          ticktext=['Весна', 'Літо', 'Осінь', 'Зима'])
        ),
        hovertemplate=(
            'Температура: %{x}<br>'
            'Вологість: %{y}<br>'
            )
    ),
    row=2, col=2
)

fig_d.update_xaxes(title_text="Година доби", row=2, col=1)
fig_d.update_yaxes(title_text="Середня кількість оренд", row=2, col=1)

fig_d.update_xaxes(title_text="Температура", row=2, col=2)
fig_d.update_yaxes(title_text="Вологість", row=2, col=2)

fig_d.update_layout(
    height=800,
    showlegend=False,
    template='plotly_white',
    title_text="Аналіз погодних умов і оренд велосипедів"
)

fig_d.show()

Розміщення графіків на одному дашборді дозволяє побачити повну картину і комплексно оцінити дані, дозволяє порівняти паттерни або виявити зв'язки між різними показниками, що в цілому підвищує швидкість і ефективність прийняття рішень.

## Завдання 5: 3D візуалізація

**Завдання:**
Створіть 3D scatter plot для аналізу взаємозв'язку температури, швидкості вітру та загальної кількості орендованих велосипедів. Колір встановіть за сезоном, а розмір - за загальною кількість оренд також.

Дайте відповіді на питання.
**Питання для інтерпретації:**
1. Яку додаткову інформацію, на ваш погляд, дає 3D візуалізація?
2. Чи видно кластери в 3D просторі?
3. Чи ви можете зробити висновки з цієї візуалізації, чи вам було простіше побудувати кілька 2D?



In [8]:
fig = px.scatter_3d(
    df, 
    x='temp', 
    y='windspeed', 
    z='count',
    color='season_name',
    size='count',
    title='<b>3D Аналіз взаємозв\'язку температури, швидкості вітру та кількості оренд велосипедів</b>',
    labels={
        'temp': 'Температура (°C)',
        'windspeed': 'Швидкість вітру',
        'count': 'Кількість оренд',
        'season_name': 'Сезон'
    },
    color_discrete_map={
        'Весна': "#FA67B3",
        'Літо': "#15B137", 
        'Осінь': "#F58719",
        'Зима': "#2950DB"
    }
)

fig.update_layout(
    title_x=0.5,
    height=800,
    scene=dict(
        xaxis=dict(
            title='Температура (°C)',
            autorange='reversed'),
        yaxis_title='Швидкість вітру',
        zaxis_title='Кількість оренд'
    )
)

fig.show()

1) За допомогою 3D візуалізації можна побачити зв'язки і виділити кластери і складні паттерни, які не є очевидними на звичайних графіках
2) На графіку видно кластери по температурі по сезонах
3) З цієї візуалізації можна зробити висновок, що найменше велосипедів орендуюсь весною (що по даним відповідає першому кварталу року), коли низька температура. Найбільше велосипеди орендують в другій половині року (осінь, зима). Швидкість вітру майже не впливає на кількість оренд, лише якщо вітер сильніше 40 м/c, то кількість оренд знижується. Також видно, що є спостереження, де швидкість вітру 0.

Я вважаю, що 3D візуалізації доцільно використовувати лише в тому випадку, коли вони дійсно можуть показати приховані складні патерни, яких не видно на звичайних графіках. В даному випадку простіше було побудувати декілька простих графіків. В даному випадку температура і швидкість вітру ніяк не корелюють і складно розпізнати паттерни і зробити висновки.



## Завдання 6: Експорт та збереження інтерактивних графіків

**Завдання:**
Збережіть побудований раніше дашборд в формат HTML. Також змініть вручну щось на дашборді (зум, виділення частини графіку) і збережіть його як статичне зображення через іконку фотоапарату у формат PNG. Завантажте файли з дашбордом у HTML та PNG (або посилання на них на github) разом з посиланням на цей ноутбук при здачі ДЗ.


In [9]:
fig_d.write_html("dashboard.html")